# Welcome to NetBeez API examples

This notebook uses a Python environment with a few libraries, including `dask`, all of which were specificied using a `conda` [environment.yml](../edit/environment.yml) file. To demo the environment, we'll show a simplified example of using `dask` to analyze time series data, adapted from Matthew Rocklin's excellent repo of [dask examples](https://github.com/blaze/dask-examples) — check out that repo for the full version (and many other examples).

## Setup your NetBeez Beezkeeper details

First you need to identify your server's FQDN and generate an API key to use in these examples.

If the URL you use to access NetBeez looks like this https://demo.netbeecloud.net then `demo.netbeecloud.net` is the fqdn.

In [1]:
beezkeeper_fqdn="demo.netbeezcloud.net"
base_url='https://' + beezkeeper_fqdn

In [2]:
from IPython.display import Markdown as md

md(f"To generate an API key head over to https://{beezkeeper_fqdn}/#settings-tab/api-keys-settings")

To generate an API key head over to https://demo.netbeezcloud.net/#settings-tab/api-keys-settings

In [3]:
api_key="7g5_hKXw_460SNKEkJEYlMn1eqw3iTmaus_cCMZ8REhc5MOEtEAx8g"

## Setup the headers
We define the content type format as JSON and provide the API token that you generated in the previous step as part of the headers.

In [4]:
import requests
import urllib3

urllib3.disable_warnings()


headers = {
            'Cache-Control': 'no-store',
            'Content-type': 'application/json',
            'Authorization': 'Bearer ' + api_key
          }

legacy_api_headers = {
    'Authorization': api_key,
    'API-VERSION': 'v1',
    'Content-type': 'application/json'
}

First search for the agent by name (https://api.netbeez.net/#f3d48709-5344-45b7-af9b-7b92feddd4ec)

# Get NetBeez Agent information and last 24 hours Ping results

First we set retrieve the agent by search by name.

In [5]:
search_name = "Rome"
url = f"{base_url}/agents?filter[name]={search_name}&type=beta"


In [6]:
response = requests.request("GET", url, headers=headers, verify=False)

import json
data = json.loads(response.text)
agent = data['data'][0]
print(agent)

{"data":[{"id":"343","type":"agent","attributes":{"name":"Rome","reached_target_test_limit":false,"reached_scheduled_test_limit":null,"nb_target_ids":[437,32],"nb_test_ids":[1698492,1698493,1698494,1698489,1698490,1698491,1698495,1698496],"agent_type_name":"rpi-model-b-3","os_version":"Raspbian GNU/Linux 9.11 (stretch)","kernel_version":"#1253 SMP Thu Aug 15 11:49:46 BST 2019","uuid":"1d2dbb0a-f034-499a-b1fd-9f27d8f0ca6c","agent_class":"faste","active":true,"active_ts":1624417551111,"description":null,"category":"network_agent","software_version":"8.0.6","open_incident_id":null,"test_aggregate_counts_by_type":{"1":{"success":2,"fail":0,"warning":0,"paused":0,"unknown":0},"2":{"success":2,"fail":0,"warning":0,"paused":0,"unknown":0},"3":{"success":2,"fail":0,"warning":0,"paused":0,"unknown":0},"4":{"success":1,"fail":0,"warning":0,"paused":0,"unknown":0}}},"relationships":{"network_interfaces":{"data":[{"id":"389","type":"network_interface"}]},"agent_groups":{"data":[{"id":"5","type":"a

Then we iterate through the tests to find the ping tests.

In [12]:
nb_tests = []
for nb_test_id in agent['attributes']['nb_test_ids']:
  url = f"{base_url}/nb_tests/{nb_test_id}.json"
  response = requests.request("GET", url, headers=legacy_api_headers, verify=False)
  data = json.loads(response.text)
  if data['heir_type'] == 'PingTest':
    print(data)
    nb_tests.append(data)


{'id': 1698492, 'target': 'www.google.com', 'count': 0, 'interval': 5, 'timeout': 5, 'start_ts': '2021-06-15T11:39:06.773-04:00', 'created_at': '2021-06-15T15:39:06.000Z', 'updated_at': '2021-06-23T03:05:52.000Z', 'agent_id': 343, 'test_type_id': 1, 'heir_id': 11483, 'heir_type': 'PingTest', 'ad_hoc': False, 'test_name': None, 'nb_test_templatable_type': 'NbTestTemplate', 'nb_test_templatable_id': 99, 'current_alert_mode': 'success', 'wifi_profile_id': None, 'network_interface_id': 389, 'nb_target_id': 32, 'nb_test_template_id': 99, 'schedule_type': 'periodic', 'options': {'data_size': 54, 'dont_fragment': False, 'tos': None, 'port': None, 'ping_type': 2, 'flags': None, 'jitter_mos': True}, 'alert_mode': 'success', 'interface_type': 'wired'}
{'id': 1698489, 'target': 'demo-app.netbeezcloud.net', 'count': 0, 'interval': 5, 'timeout': 5, 'start_ts': '2021-06-15T11:39:06.605-04:00', 'created_at': '2021-06-15T15:39:06.000Z', 'updated_at': '2021-06-23T03:05:52.000Z', 'agent_id': 343, 'test_

Then we retrieve the results for the past 24 hours.

In [17]:
import time
import datetime
to_ts = int(time.time() * 1000)
from_ts = to_ts - (60 * 60 * 1000)

print(datetime.datetime.fromtimestamp(from_ts/1000.0))
print(datetime.datetime.fromtimestamp(to_ts/1000.0))


2021-06-23 14:34:12.957000
2021-06-23 15:34:12.957000


In [25]:
results = []
import pandas as pd
for nb_test in nb_tests:
  url = f"{base_url}/results.json?nb_test_id={nb_test['id']}&from={from_ts}&to={to_ts}"
  response = requests.request("GET", url, headers=legacy_api_headers, verify=False)
  results.append(pd.json_normalize(json.loads(response.text), 'index'))

In [26]:
results[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719 entries, 0 to 718
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               719 non-null    int64  
 1   value            719 non-null    float64
 2   timestamp        719 non-null    int64  
 3   sequence_number  719 non-null    int64  
 4   nb_test_id       719 non-null    int64  
dtypes: float64(1), int64(4)
memory usage: 28.2 KB
